# Homework 4

In this homework, you will writing two functions that will assign a final letter grade to each student.  The scores of each student on each assignment are given in "Final_Grades.csv". In this semester, there 6 homeworks (columns labeled HW_1, .., HW_6), 3 participation scores (column Part_1, Part_2, Part_3) and a midterm and final.

## Problem 1
Your first task is to write a function callled "Get_Final_Perc" that has the following three inputs:

- HW_weight: The weight towards the final grade of the score on all homeworks.
- Exam_weight: The weight towards the final grade of the midterm and final. You may exam the midterm and final are weighted equally
- Part_weight: The weight towards the final grade of the score on all participation scores.

IMPORTANT:  For the HW and participation categories, the final percentage is (# points earned)/(# of points possible).  So, for example, if there were two HWs and you got 10/10 and 7/8, your final percentage on HW would be 17/18.

Inside of this function, you should

- Read in the data set into a dataframe called df_grades.
- Add to df_grades a column labeled "Final_perc", which gives the final percentage (this should be between 0 and 100 for each student) achieved by each student given the specified weighting scheme that is given as input.  Note that it may be helpful to add other columns along the way, this is okay.


A few other things that you must keep in mind when writing this function

- You should only consider students who have a score for all assignments (you can delete the rows corresponding to students with missing scores for any assignment). Hint: Recall that blanks in a csv are automatically read in as NAs.
- You may assume that for each assignment, there was at least one student who received 100% on the assignment.  Furthermore, there was no extra credit on any assignment. Hint: this is how you can figure out the max score on each assignment.

 


In [15]:
#Run this cell for your imports
import pandas as pd
import numpy as np

def Get_Final_Perc(HW_weight, Exam_weight, Part_weight):
    
    df_grades = pd.read_csv("./Data/Final_Grades.csv")
    
    ### BEGIN SOLUTION
    df_grades.dropna(inplace = True)
    max_scores = df_grades.max()
    assigments = df_grades.columns
    HW_columns = [c for c in assigments if "HW" in c]
    Part_columns = [c for c in assigments if "Part" in c]
    df_grades["HW_perc"] = df_grades[HW_columns].sum(axis=1)/max_scores[HW_columns].sum()
    df_grades["Part_perc"] = df_grades[Part_columns].sum(axis=1)/max_scores[Part_columns].sum()
    df_grades["Midterm_perc"] = df_grades.Midterm/max_scores["Midterm"]
    df_grades["Final_perc"] = df_grades.Final/max_scores["Final"]
    df_grades["Final_perc"] = df_grades.HW_perc*HW_weight + (df_grades.Midterm_perc + \
                                                             df_grades.Final_perc)*Exam_weight+\
                                                            df_grades.Part_perc*Part_weight
    ### END SOLUTION
    
    
    return df_grades

Get_Final_Perc(50, 25, 0).mean()

HW_5            29.294737
HW_6            29.400000
Final           43.931579
HW_1            19.410526
Part_1           0.905263
HW_2            29.326316
Part_2           0.936842
HW_3            19.378947
Part_3           0.894737
Midterm         35.094737
HW_4            15.431579
HW_perc          0.974261
Part_perc        0.912281
Midterm_perc     0.779883
Final_perc      89.331077
dtype: float64

In [2]:
assert np.isclose(Get_Final_Perc(40, 25, 10).Final_perc.mean(),88.711)
### BEGIN HIDDEN TESTS
assert np.isclose(Get_Final_Perc(50, 25, 0).Final_perc.mean(),89.331)
### END HIDDEN TESTS

In [3]:
assert np.isclose(Get_Final_Perc(100, 0, 0).Final_perc.mean(),97.426)
### BEGIN HIDDEN TESTS
assert Get_Final_Perc(0, 0, 0).shape == (95, 16)
### END HIDDEN TESTS

In [4]:
assert np.isclose(Get_Final_Perc(1, 49, 1).Final_perc.mean(),81.4978754798834)

## Problem 2
Your second and final task is to write a function called "Assign_Grades", which as the following two inputs:

- df_grades_w_final_perc: This is the df_grades dataframe with the Final_Perc column added to it.  In other words, you may assume that df_grades_w_final_perc has a column called Final_perc which is each student's final percentage.

- cutoffs: This is a list that always as 4 entries.  The first entry is the minimum score needed for an "A", the second entry is the minumum score needed for a "B", the third entry is the minumum score needed for a "C" and the last entry is the miminum score need for a "D". Any score below this fourth entry is awarded an "F".  You may assume that all cutoffs are unique so you don't have to worry about ties. Finally, you may assume that the cutoffs for better grades are always higher than the cutoffs for lower grades.

You goal is to add to df_grades_w_final_perc a column called "letter", which contains each student's letter grade (as a string, i.e. an A should added as "A") according to the inputted cutoff points.

Hint: The pandas method "cut" could be very helpful here.

In [5]:
def Assign_Grades(df_grades_w_final_perc, cutoffs):
    
    ### BEGIN SOLUTION
    grade_cutoffs = [0] + [cutoffs[-1 -i] for i in range(len(cutoffs))] + [110]
    labels = ["F","D","C","B", "A" ]
    df_grades_w_final_perc["letter"] = pd.cut(df_grades_w_final_perc.Final_perc,\
                                              bins = grade_cutoffs, labels = labels, right=False)

    ### END SOLUTION
    return df_grades_w_final_perc.letter.value_counts()

In [6]:
df_grades = Get_Final_Perc(40, 25, 10)
cutoffs = [90,80,70,60]
assert Assign_Grades(df_grades, cutoffs)["A"]==49

### BEGIN HIDDEN TESTS
df_grades = Get_Final_Perc(40, 25, 10)
cutoffs = [95,85,75,65]
assert Assign_Grades(df_grades, cutoffs)["A"] == 13
### END HIDDEN TESTS

In [7]:
df_grades = Get_Final_Perc(100, 0, 0)
cutoffs = [100,80,70,60]
assert Assign_Grades(df_grades, cutoffs)["A"]==27